---
## 1. Setup and Data Loading

In [1]:
# ============================================
# IMPORT LIBRARIES
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

print("✅ Libraries imported")

✅ Libraries imported


In [2]:
# ============================================
# LOAD RAW DATA
# ============================================

df = pd.read_csv('../data/raw/Output_10yrs_reported_schools_17220.csv')

# Standardize column names
df = df.rename(columns={
    'Survey Year': 'Year',
    'Institution Name': 'Institution_Name',
    'State CD': 'State',
    'Classification Name': 'Classification'
})

# CRITICAL: Sort by institution and year for correct lag calculations
df = df.sort_values(['UNITID', 'Year']).reset_index(drop=True)

print(f"✅ Loaded raw data: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"📅 Years: {df['Year'].min()} - {df['Year'].max()}")
print(f"🏫 Institutions: {df['UNITID'].nunique():,}")

✅ Loaded raw data: 17,220 rows × 580 columns
📅 Years: 2014 - 2023
🏫 Institutions: 1,722


---
## 2. Year Allocation Reminder

Before we start, let's be clear about how years will be used:

In [3]:
# ============================================
# YEAR ALLOCATION DOCUMENTATION
# ============================================

year_info = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                              YEAR ALLOCATION                                 ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  Raw Data: 2014-2023 (10 years, 17,220 records)                              ║
║                                                                              ║
║  After Feature Engineering:                                                  ║
║  ─────────────────────────────                                               ║
║  • 2014-2016: CONSUMED for 2-year lag calculations                           ║
║               (not in final dataset, used as lookback)                       ║
║                                                                              ║
║  • 2017-2022: USABLE ROWS (have both features AND targets)                   ║
║               These will be split into train/val/test                        ║
║                                                                              ║
║  • 2023:      FUTURE PREDICTION (has features, NO target)                    ║
║               Target requires 2024 data which doesn't exist                  ║
║                                                                              ║
║  Expected Row Counts:                                                        ║
║  ─────────────────────                                                       ║
║  • 2017-2022 with targets: ~1,722 × 6 = ~10,332 rows                         ║
║  • 2023 without targets: ~1,722 rows                                         ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(year_info)


╔══════════════════════════════════════════════════════════════════════════════╗
║                              YEAR ALLOCATION                                 ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  Raw Data: 2014-2023 (10 years, 17,220 records)                              ║
║                                                                              ║
║  After Feature Engineering:                                                  ║
║  ─────────────────────────────                                               ║
║  • 2014-2016: CONSUMED for 2-year lag calculations                           ║
║               (not in final dataset, used as lookback)                       ║
║                                                                              ║
║  • 2017-2022: USABLE ROWS (have both features AND targets)                   ║
║               These will 

---
## 3. Build Structural Features

These features don't involve time calculations.

In [4]:
# ============================================
# FEATURE: Division
# ============================================
# Based on: EDA Observation 2 - Huge differences between D1, D2, D3
# Why useful: D1 has 10x revenue of D2; fundamentally different profiles
# Why safe: Structural attribute, doesn't change, no future info

def extract_division(classification):
    """Extract D1/D2/D3 from full classification name"""
    if pd.isna(classification):
        return 'Unknown'
    if 'NCAA Division I' in classification:
        return 'D1'
    elif 'NCAA Division II' in classification:
        return 'D2'
    elif 'NCAA Division III' in classification:
        return 'D3'
    else:
        return 'Other'

df['Division'] = df['Classification'].apply(extract_division)

print("✅ Division feature created")
print(df['Division'].value_counts())

✅ Division feature created
Division
D1         9888
Other      7176
Unknown     156
Name: count, dtype: int64


In [5]:
# ============================================
# FEATURE: State (already exists as 'State')
# ============================================
# Based on: General - Regional patterns may exist
# Why useful: State funding policies vary
# Why safe: Location doesn't change, no future info

print("✅ State feature available")
print(f"Unique states: {df['State'].nunique()}")

✅ State feature available
Unique states: 51


---
## 4. Build Current Year Metrics

In [6]:
# ============================================
# FEATURE: Total_Athletes
# ============================================
# Based on: EDA Observation 5 - Participation correlates with financials
# Formula: Men's + Women's participation
# Why useful: Program size indicator
# Why safe: Current year value only

df['Total_Athletes'] = (
    df["Unduplicated Count Men's Participation"].fillna(0) + 
    df["Unduplicated Count Women's Participation"].fillna(0)
)

print("✅ Total_Athletes feature created")
print(f"   Mean: {df['Total_Athletes'].mean():.0f}")
print(f"   Max:  {df['Total_Athletes'].max():.0f}")

✅ Total_Athletes feature created
   Mean: 307
   Max:  1416


In [7]:
# ============================================
# FEATURE: Efficiency_Ratio
# ============================================
# Based on: EDA Observation 4 - Rev/Exp ratio captures financial health
# Formula: Revenue[t] / Expenses[t]
# Why useful: >1 = surplus, <1 = deficit
# Why safe: Uses only current year data

df['Efficiency_Ratio'] = df['Grand Total Revenue'] / df['Grand Total Expenses'].replace(0, np.nan)

print("✅ Efficiency_Ratio feature created")
print(f"   Mean:   {df['Efficiency_Ratio'].mean():.3f}")
print(f"   Median: {df['Efficiency_Ratio'].median():.3f}")

✅ Efficiency_Ratio feature created
   Mean:   1.027
   Median: 1.000


In [8]:
# ============================================
# FEATURE: Revenue_Per_Athlete
# ============================================
# Based on: EDA Observation 5 - Normalized metrics are useful
# Formula: Revenue[t] / Total_Athletes[t]
# Why useful: Comparable across institutions of different sizes
# Why safe: Uses only current year data

df['Revenue_Per_Athlete'] = df['Grand Total Revenue'] / df['Total_Athletes'].replace(0, np.nan)

print("✅ Revenue_Per_Athlete feature created")
print(f"   Mean:   ${df['Revenue_Per_Athlete'].mean():,.0f}")
print(f"   Median: ${df['Revenue_Per_Athlete'].median():,.0f}")

✅ Revenue_Per_Athlete feature created
   Mean:   $27,910
   Median: $12,575


In [9]:
# ============================================
# FEATURE: Reports_Exactly_One
# ============================================
# Based on: EDA Observation 6 - Many report exactly 1.0 efficiency
# Formula: 1 if Efficiency_Ratio == 1.0 else 0
# Why useful: Flags potential accounting anomaly
# Why safe: Uses only current year data

df['Reports_Exactly_One'] = (df['Efficiency_Ratio'] == 1.0).astype(int)

print("✅ Reports_Exactly_One flag created")
print(f"   Institutions reporting exactly 1.0: {df['Reports_Exactly_One'].sum():,} ({df['Reports_Exactly_One'].mean()*100:.1f}%)")

✅ Reports_Exactly_One flag created
   Institutions reporting exactly 1.0: 10,315 (59.9%)


---
## 5. Build Growth Features (1-Year Lookback)

These features use data from the PREVIOUS year (safe).

In [10]:
# ============================================
# GROUP BY INSTITUTION FOR LAG CALCULATIONS
# ============================================
# We group by UNITID so that lag calculations are within each institution
# shift(1) means: look at the PREVIOUS year's value (SAFE)
# shift(-1) would mean: look at NEXT year (FORBIDDEN for features)

grouped = df.groupby('UNITID')

print("✅ Grouped by institution for lag calculations")

✅ Grouped by institution for lag calculations


In [11]:
# ============================================
# FEATURE: Revenue_Growth_1yr
# ============================================
# Based on: EDA Observations 1 & 3 - Growth rates normalize across scales
# Formula: (Revenue[t] - Revenue[t-1]) / Revenue[t-1]
# Why useful: Captures recent momentum
# Why safe: Uses t and t-1 only (PAST data)

# Get previous year's revenue using shift(1) - this is SAFE
df['Revenue_Prev'] = grouped['Grand Total Revenue'].shift(1)
df['Revenue_Growth_1yr'] = (df['Grand Total Revenue'] - df['Revenue_Prev']) / df['Revenue_Prev']

# Clean up temporary column
df = df.drop(columns=['Revenue_Prev'])

print("✅ Revenue_Growth_1yr feature created")
print(f"   Mean:   {df['Revenue_Growth_1yr'].mean()*100:.2f}%")
print(f"   Median: {df['Revenue_Growth_1yr'].median()*100:.2f}%")

✅ Revenue_Growth_1yr feature created
   Mean:   8.12%
   Median: 4.80%


In [12]:
# ============================================
# FEATURE: Expense_Growth_1yr
# ============================================
# Based on: EDA Observation 3 - Expense growth patterns matter
# Formula: (Expenses[t] - Expenses[t-1]) / Expenses[t-1]
# Why useful: High expense growth could signal trouble
# Why safe: Uses t and t-1 only (PAST data)

df['Expense_Prev'] = grouped['Grand Total Expenses'].shift(1)
df['Expense_Growth_1yr'] = (df['Grand Total Expenses'] - df['Expense_Prev']) / df['Expense_Prev']

df = df.drop(columns=['Expense_Prev'])

print("✅ Expense_Growth_1yr feature created")
print(f"   Mean:   {df['Expense_Growth_1yr'].mean()*100:.2f}%")
print(f"   Median: {df['Expense_Growth_1yr'].median()*100:.2f}%")

✅ Expense_Growth_1yr feature created
   Mean:   7.99%
   Median: 4.89%


---
## 6. Build Trend Features (2-Year Lookback)

These features use data from the past 2 years (safe).

In [13]:
# ============================================
# FEATURE: Revenue_CAGR_2yr
# ============================================
# Based on: EDA Observation 3 - Multi-year trends more stable
# Formula: (Revenue[t] / Revenue[t-2])^0.5 - 1
# Why useful: Compound annual growth rate; smoother than 1-year
# Why safe: Uses t and t-2 only (PAST data)

df['Revenue_2yr_ago'] = grouped['Grand Total Revenue'].shift(2)
df['Revenue_CAGR_2yr'] = (df['Grand Total Revenue'] / df['Revenue_2yr_ago']) ** 0.5 - 1

df = df.drop(columns=['Revenue_2yr_ago'])

print("✅ Revenue_CAGR_2yr feature created")
print(f"   Mean:   {df['Revenue_CAGR_2yr'].mean()*100:.2f}%")
print(f"   Median: {df['Revenue_CAGR_2yr'].median()*100:.2f}%")

✅ Revenue_CAGR_2yr feature created
   Mean:   6.35%
   Median: 4.80%


In [14]:
# ============================================
# FEATURE: Expense_CAGR_2yr
# ============================================
# Based on: EDA Observation 3 - Expense trajectory matters
# Formula: (Expenses[t] / Expenses[t-2])^0.5 - 1
# Why useful: Compound expense growth signals spending trajectory
# Why safe: Uses t and t-2 only (PAST data)

df['Expense_2yr_ago'] = grouped['Grand Total Expenses'].shift(2)
df['Expense_CAGR_2yr'] = (df['Grand Total Expenses'] / df['Expense_2yr_ago']) ** 0.5 - 1

df = df.drop(columns=['Expense_2yr_ago'])

print("✅ Expense_CAGR_2yr feature created")
print(f"   Mean:   {df['Expense_CAGR_2yr'].mean()*100:.2f}%")
print(f"   Median: {df['Expense_CAGR_2yr'].median()*100:.2f}%")

✅ Expense_CAGR_2yr feature created
   Mean:   6.27%
   Median: 4.82%


In [15]:
# ============================================
# FEATURE: Efficiency_Mean_2yr
# ============================================
# Based on: EDA Observation 4 - Efficiency over time
# Formula: mean(Efficiency[t], Efficiency[t-1])
# Why useful: Smoothed efficiency reduces noise
# Why safe: Uses t and t-1 only (PAST data)

df['Efficiency_Mean_2yr'] = grouped['Efficiency_Ratio'].transform(
    lambda x: x.rolling(window=2, min_periods=2).mean()
)

print("✅ Efficiency_Mean_2yr feature created")
print(f"   Mean:   {df['Efficiency_Mean_2yr'].mean():.3f}")
print(f"   Median: {df['Efficiency_Mean_2yr'].median():.3f}")

✅ Efficiency_Mean_2yr feature created
   Mean:   1.028
   Median: 1.000


---
## 7. Build Volatility Features

In [16]:
# ============================================
# FEATURE: Revenue_Volatility_2yr
# ============================================
# Based on: EDA Observation 7 - Volatility varies by institution
# Formula: std(Revenue[t], Revenue[t-1])
# Why useful: High volatility = unstable finances
# Why safe: Uses t and t-1 only (PAST data)

df['Revenue_Volatility_2yr'] = grouped['Grand Total Revenue'].transform(
    lambda x: x.rolling(window=2, min_periods=2).std()
)

print("✅ Revenue_Volatility_2yr feature created")

✅ Revenue_Volatility_2yr feature created


In [17]:
# ============================================
# FEATURE: Expense_Volatility_2yr
# ============================================
# Based on: EDA Observation 7 - Volatility captures stability
# Formula: std(Expenses[t], Expenses[t-1])
# Why useful: Expense volatility signals unpredictable spending
# Why safe: Uses t and t-1 only (PAST data)

df['Expense_Volatility_2yr'] = grouped['Grand Total Expenses'].transform(
    lambda x: x.rolling(window=2, min_periods=2).std()
)

print("✅ Expense_Volatility_2yr feature created")

✅ Expense_Volatility_2yr feature created


---
## 8. Create Target Variable

**Note:** The target uses `shift(-1)` because we're looking at what happens NEXT year. This is correct for targets, but would be forbidden for features.

In [18]:
# ============================================
# TARGET: Financial_Trajectory
# ============================================
# Definition: Predict trajectory from year t to year t+1
# 
# IMPORTANT: We use shift(-1) here because the TARGET should look at
# future outcomes. This is the ONE place where looking ahead is valid.
#
# Classes:
# - Improving (2): Rev Growth > 3% AND Exp Growth < Rev Growth
# - Declining (0): Rev Growth < 0% OR Exp Growth > Rev Growth + 3%
# - Stable (1): Everything else

# Calculate FUTURE growth (what happens from t to t+1)
# shift(-1) means: look at NEXT year's value
df['Future_Revenue'] = grouped['Grand Total Revenue'].shift(-1)
df['Future_Expenses'] = grouped['Grand Total Expenses'].shift(-1)

df['Future_Revenue_Growth'] = (df['Future_Revenue'] - df['Grand Total Revenue']) / df['Grand Total Revenue']
df['Future_Expense_Growth'] = (df['Future_Expenses'] - df['Grand Total Expenses']) / df['Grand Total Expenses']

# Define trajectory classification function
def classify_trajectory(row):
    """Classify financial trajectory based on future growth."""
    rev_growth = row['Future_Revenue_Growth']
    exp_growth = row['Future_Expense_Growth']
    
    if pd.isna(rev_growth) or pd.isna(exp_growth):
        return np.nan
    
    # IMPROVING: Strong revenue growth AND expenses under control
    if (rev_growth > 0.03) and (exp_growth < rev_growth):
        return 'Improving'
    
    # DECLINING: Revenue shrinking OR expenses outpacing revenue
    elif (rev_growth < 0.00) or (exp_growth > rev_growth + 0.03):
        return 'Declining'
    
    # STABLE: Everything else
    else:
        return 'Stable'

df['Target_Trajectory'] = df.apply(classify_trajectory, axis=1)

# Numeric encoding
trajectory_map = {'Declining': 0, 'Stable': 1, 'Improving': 2}
df['Target_Label'] = df['Target_Trajectory'].map(trajectory_map)

# Clean up intermediate columns
df = df.drop(columns=['Future_Revenue', 'Future_Expenses', 'Future_Revenue_Growth', 'Future_Expense_Growth'])

print("✅ Target variable created")
print(df['Target_Trajectory'].value_counts())

✅ Target variable created
Target_Trajectory
Stable       7513
Declining    5809
Improving    2176
Name: count, dtype: int64


---
## 9. Filter to Usable Rows

We need to remove rows that:
1. Don't have enough history for 2-year features (2014-2016)
2. Don't have a target (2023)

In [19]:
# ============================================
# FILTER ROWS
# ============================================

print("📊 BEFORE FILTERING:")
print(f"   Total rows: {len(df):,}")
print(f"   Year range: {df['Year'].min()} - {df['Year'].max()}")

# Rows with valid 2-year features (need Revenue_CAGR_2yr to be non-null)
has_features = df['Revenue_CAGR_2yr'].notna()
print(f"   Rows with valid features: {has_features.sum():,}")

# Rows with valid target
has_target = df['Target_Label'].notna()
print(f"   Rows with valid target: {has_target.sum():,}")

# Rows usable for training (have BOTH features AND target)
usable_for_training = has_features & has_target
print(f"   Rows usable for training: {usable_for_training.sum():,}")

# Rows usable for prediction (have features but NO target - 2023)
usable_for_prediction = has_features & ~has_target
print(f"   Rows for future prediction: {usable_for_prediction.sum():,}")

📊 BEFORE FILTERING:
   Total rows: 17,220
   Year range: 2014 - 2023
   Rows with valid features: 13,776
   Rows with valid target: 15,498
   Rows usable for training: 12,054
   Rows for future prediction: 1,722


In [20]:
# ============================================
# CREATE TRAINING AND PREDICTION DATASETS
# ============================================

# Training data: rows with both features and target (2017-2022)
df_train = df[usable_for_training].copy()

# Prediction data: rows with features but no target (2023)
df_predict = df[usable_for_prediction].copy()

print(f"\n✅ DATASETS CREATED:")
print(f"   Training dataset: {len(df_train):,} rows")
print(f"   Years in training: {df_train['Year'].min()} - {df_train['Year'].max()}")
print(f"   \n   Prediction dataset: {len(df_predict):,} rows")
print(f"   Years in prediction: {df_predict['Year'].unique()}")


✅ DATASETS CREATED:
   Training dataset: 12,054 rows
   Years in training: 2016 - 2022
   
   Prediction dataset: 1,722 rows
   Years in prediction: [2023]


---
## 10. ⚠️ CRITICAL: Leakage Verification

Before saving, we MUST verify there's no data leakage.

In [21]:
# ============================================
# LEAKAGE CHECK 1: No Target-Derived Features
# ============================================

print("🔍 LEAKAGE CHECK 1: Checking for target-derived features...")
print("="*60)

# List all columns
all_columns = df_train.columns.tolist()

# Check for any column containing 'Target' except the actual target columns
target_related = [col for col in all_columns if 'Target' in col]
allowed_target_cols = ['Target_Trajectory', 'Target_Label']
forbidden_target_cols = [col for col in target_related if col not in allowed_target_cols]

if len(forbidden_target_cols) == 0:
    print("✅ PASS: No target-derived features found in dataset")
else:
    print("❌ FAIL: Found target-derived features:")
    for col in forbidden_target_cols:
        print(f"   - {col}")
    print("   These MUST be removed before training!")

🔍 LEAKAGE CHECK 1: Checking for target-derived features...
✅ PASS: No target-derived features found in dataset


In [22]:
# ============================================
# LEAKAGE CHECK 2: Verify Feature Columns
# ============================================

print("\n🔍 LEAKAGE CHECK 2: Listing all feature columns...")
print("="*60)

# Define expected feature columns
identifier_cols = ['UNITID', 'Institution_Name', 'Year', 'State', 'Classification']
target_cols = ['Target_Trajectory', 'Target_Label']

feature_cols = [col for col in all_columns if col not in identifier_cols + target_cols]

print("FEATURE COLUMNS (will be used for training):")
for i, col in enumerate(feature_cols, 1):
    print(f"   {i:2}. {col}")

print(f"\nTotal features: {len(feature_cols)}")


🔍 LEAKAGE CHECK 2: Listing all feature columns...
FEATURE COLUMNS (will be used for training):
    1. OPE ID
    2. Classification Other
    3. Sanction Code
    4. Sanction Name
    5. Male Undergraduates
    6. Female Undergraduates
    7. Total Undergraduates
    8. Archery Men's Team Participation
    9. Archery Women's Team Participation
   10. Archery Coed Team Men Participation
   11. Archery Coed Team Women Participation
   12. Archery Total Participation
   13. Badminton Women's Team Participation
   14. Badminton Total Participation
   15. Baseball Men's Team Participation
   16. Baseball Total Participation
   17. Basketball Men's Team Participation
   18. Basketball Women's Team Participation
   19. Basketball Coed Team Men Participation
   20. Basketball Coed Team Women Participation
   21. Basketball Total Participation
   22. Beach Volleyball Men's Team Participation
   23. Beach Volleyball Women's Team Participation
   24. Beach Volleyball Total Participation
   25. Bo

In [23]:
# ============================================
# LEAKAGE CHECK 3: No Future-Looking Features
# ============================================

print("\n🔍 LEAKAGE CHECK 3: Checking for future-looking features...")
print("="*60)

# Check for any column containing 'Future'
future_cols = [col for col in all_columns if 'Future' in col]

if len(future_cols) == 0:
    print("✅ PASS: No future-looking features found")
else:
    print("❌ FAIL: Found future-looking features:")
    for col in future_cols:
        print(f"   - {col}")

# Check for any 'Lag1_Target' or similar
lag_target_cols = [col for col in all_columns if 'Lag' in col and 'Target' in col]

if len(lag_target_cols) == 0:
    print("✅ PASS: No lagged target features found")
else:
    print("❌ FAIL: Found lagged target features:")
    for col in lag_target_cols:
        print(f"   - {col}")


🔍 LEAKAGE CHECK 3: Checking for future-looking features...
✅ PASS: No future-looking features found
✅ PASS: No lagged target features found


In [24]:
# ============================================
# LEAKAGE CHECK SUMMARY
# ============================================

print("\n" + "="*60)
print("📋 LEAKAGE CHECK SUMMARY")
print("="*60)

checks_passed = (
    len(forbidden_target_cols) == 0 and
    len(future_cols) == 0 and
    len(lag_target_cols) == 0
)

if checks_passed:
    print("\n✅ ALL CHECKS PASSED - Dataset is leak-free!")
    print("\nSafe to proceed with temporal split and model training.")
else:
    print("\n❌ CHECKS FAILED - Dataset has potential leakage!")
    print("\nDO NOT proceed until issues are fixed.")


📋 LEAKAGE CHECK SUMMARY

✅ ALL CHECKS PASSED - Dataset is leak-free!

Safe to proceed with temporal split and model training.


---
## 11. Select Final Columns and Save

In [25]:
# ============================================
# SELECT FINAL COLUMNS
# ============================================

# Columns to keep in final dataset
final_columns = [
    # Identifiers (kept for reference, not used in model)
    'UNITID', 'Institution_Name', 'Year', 'State',
    
    # Structural features
    'Division',
    
    # Current metrics
    'Grand Total Revenue', 'Grand Total Expenses', 'Total_Athletes',
    'Efficiency_Ratio', 'Revenue_Per_Athlete', 'Reports_Exactly_One',
    
    # Growth features (1-year)
    'Revenue_Growth_1yr', 'Expense_Growth_1yr',
    
    # Trend features (2-year)
    'Revenue_CAGR_2yr', 'Expense_CAGR_2yr', 'Efficiency_Mean_2yr',
    
    # Volatility features
    'Revenue_Volatility_2yr', 'Expense_Volatility_2yr',
    
    # Target (only for training data)
    'Target_Trajectory', 'Target_Label'
]

# Filter to only existing columns
final_columns = [col for col in final_columns if col in df_train.columns]

df_train_final = df_train[final_columns].copy()

# For prediction data, exclude target columns
pred_columns = [col for col in final_columns if col not in ['Target_Trajectory', 'Target_Label']]
df_predict_final = df_predict[pred_columns].copy()

print(f"✅ Final training dataset: {df_train_final.shape}")
print(f"✅ Final prediction dataset: {df_predict_final.shape}")

✅ Final training dataset: (12054, 20)
✅ Final prediction dataset: (1722, 18)


In [26]:
# ============================================
# SAVE DATASETS
# ============================================

# Save training data (2017-2022 with targets)
train_path = '../data/processed/features_temporal.csv'
df_train_final.to_csv(train_path, index=False)
print(f"✅ Saved training data to: {train_path}")

# Save prediction data (2023 without targets)
predict_path = '../data/processed/features_2023_predict.csv'
df_predict_final.to_csv(predict_path, index=False)
print(f"✅ Saved prediction data to: {predict_path}")

✅ Saved training data to: ../data/processed/features_temporal.csv
✅ Saved prediction data to: ../data/processed/features_2023_predict.csv


---
## 12. Final Summary

In [27]:
# ============================================
# FEATURE ENGINEERING SUMMARY
# ============================================

summary = f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                     FEATURE ENGINEERING COMPLETE                             ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  TRAINING DATA (features_temporal.csv):                                      ║
║  ─────────────────────────────────────────                                   ║
║  • Rows: {len(df_train_final):,}                                                         ║
║  • Years: {df_train_final['Year'].min()} - {df_train_final['Year'].max()}                                                        ║
║  • Institutions: {df_train_final['UNITID'].nunique():,}                                                    ║
║  • Features: {len(final_columns) - 6} (excluding identifiers and target)                            ║
║                                                                              ║
║  PREDICTION DATA (features_2023_predict.csv):                                ║
║  ─────────────────────────────────────────────                               ║
║  • Rows: {len(df_predict_final):,}                                                          ║
║  • Year: {df_predict_final['Year'].unique()[0] if len(df_predict_final) > 0 else 'N/A'}                                                             ║
║  • No target (true future prediction)                                        ║
║                                                                              ║
║  CLASS DISTRIBUTION (Training):                                              ║
║  ─────────────────────────────────                                           ║
║  • Declining: {(df_train_final['Target_Trajectory'] == 'Declining').sum():,} ({(df_train_final['Target_Trajectory'] == 'Declining').mean()*100:.1f}%)                                            ║
║  • Stable:    {(df_train_final['Target_Trajectory'] == 'Stable').sum():,} ({(df_train_final['Target_Trajectory'] == 'Stable').mean()*100:.1f}%)                                            ║
║  • Improving: {(df_train_final['Target_Trajectory'] == 'Improving').sum():,} ({(df_train_final['Target_Trajectory'] == 'Improving').mean()*100:.1f}%)                                            ║
║                                                                              ║
║  LEAKAGE STATUS: ✅ VERIFIED CLEAN                                           ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

NEXT STEP: Proceed to 05_Temporal_Split.ipynb
→ Split data by time: train (2017-2019), val (2020-2021), holdout (2022)
"""

print(summary)


╔══════════════════════════════════════════════════════════════════════════════╗
║                     FEATURE ENGINEERING COMPLETE                             ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  TRAINING DATA (features_temporal.csv):                                      ║
║  ─────────────────────────────────────────                                   ║
║  • Rows: 12,054                                                         ║
║  • Years: 2016 - 2022                                                        ║
║  • Institutions: 1,722                                                    ║
║  • Features: 14 (excluding identifiers and target)                            ║
║                                                                              ║
║  PREDICTION DATA (features_2023_predict.csv):                                ║
║  ───────────────────────────────

---
## ✅ Checkpoint Complete

We've built:
1. **15 features** across 6 categories (all leak-free)
2. **Target variable** with 3 classes
3. **Training dataset**: 2017-2022 (~10,332 rows)
4. **Prediction dataset**: 2023 (~1,722 rows)

**Leakage verification passed** — no target-derived or future-looking features.

**Next Notebook:** `05_Temporal_Split.ipynb`  
We'll split the training data by time to ensure proper temporal validation.